# Group Varaibles into Screens

Takes in a list of AL variables and spits out suggested groupings. Here's what's going on:

1. It reads in a list of fields (e.g., `["user_name","user_address"]`)
2. Splits each field into words (e.g., turning `user_name` into `user name`)
3. It then turns these ngrams/"sentences" into vectors using word2vec. 
4. For the collection of fields, it finds clusters of these "sentences" within the semantic space defined by word2vec. Currently it uses Affinity Propagation. See https://machinelearningmastery.com/clustering-algorithms-with-python/

Load libraries and functions.

In [22]:
import re
import spacy
import numpy as np
from numpy import unique
from numpy import where
from sklearn.cluster import AffinityPropagation
from sklearn.metrics.pairwise import cosine_similarity

Before you run this next cell, make sure you have the referenced model downloaded. See https://stackoverflow.com/a/58855787 

In [2]:
nlp = spacy.load('en_core_web_lg') # this takes a while to load

In [3]:
def reCase(text):
    # a quick and dirty way to pull words out of
    # snake_case, camelCase and the like.    
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text.strip())
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [68]:
def cluster_screens(fields=[],damping=0.9):
    # Takes in a list (fields) and returns a suggested screen grouping
    # Set damping to value >= 0.5 or < 1 to tune how related screens should be
    
    vec_mat = np.zeros([len(fields),300])
    for i in range(len(fields)):
        vec_mat[i] = [nlp(reCase(fields[i])).vector][0]

    # create model
    model = AffinityPropagation(damping=damping,random_state=4)
    # fit the model
    model.fit(vec_mat)
    # assign a cluster to each example
    yhat = model.predict(vec_mat)
    # retrieve unique clusters
    clusters = unique(yhat)

    screens = {}
    #sim = np.zeros([5,300])
    i=0
    for cluster in clusters:
        this_screen = where(yhat == cluster)[0]
        vars = []
        j=0
        for screen in this_screen:
            #sim[screen]=vec_mat[screen] # use this spot to add up vectors for compare to list
            vars.append(fields[screen])
            j+=1
        screens["screen_%s"%i]=vars
        i+=1

    return screens

## Try it out

Use daming >= 0.5 or < 1 to tune the groupings.

In [69]:
fields= [
        "users1_name",
        "users1_birthdate",
        "users1_address_line_one",
        "users1_address_line_two",
        "users1_address_city",
        "users1_address_state",
        "users1_address_zip",
        "users1_phone_number",
        "users1_email",
        "plantiffs1_name",
        "defendants1_name",
        "petitioners1_name",
        "respondents1_name",
        "docket_number",
        "trial_court_county",
        "users1_signature",
        "signature_date"
        ]

cluster_screens(fields,damping=0.7)

{'screen_0': ['users1_name',
  'users1_birthdate',
  'users1_address_line_one',
  'users1_address_line_two',
  'users1_address_city',
  'users1_address_state',
  'users1_address_zip',
  'users1_phone_number',
  'users1_email',
  'users1_signature'],
 'screen_1': ['plantiffs1_name',
  'defendants1_name',
  'petitioners1_name',
  'respondents1_name'],
 'screen_2': ['docket_number'],
 'screen_3': ['trial_court_county'],
 'screen_4': ['signature_date']}

In [67]:
parts = np.zeros([5,300])

for row in mat:
    sim = []
    for part in parts:
        sim.append(cosine_similarity(mat[0].reshape(1, -1),row.reshape(1, -1))[0][0])
    print(sim)

[1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002]
[0.8038686975124008, 0.8038686975124008, 0.8038686975124008, 0.8038686975124008, 0.8038686975124008]
[0.8481956135581707, 0.8481956135581707, 0.8481956135581707, 0.8481956135581707, 0.8481956135581707]
[0.826921329803093, 0.826921329803093, 0.826921329803093, 0.826921329803093, 0.826921329803093]
[0.8197875020998185, 0.8197875020998185, 0.8197875020998185, 0.8197875020998185, 0.8197875020998185]
[0.8059375150874605, 0.8059375150874605, 0.8059375150874605, 0.8059375150874605, 0.8059375150874605]
[0.7847333680991196, 0.7847333680991196, 0.7847333680991196, 0.7847333680991196, 0.7847333680991196]
[0.8077588614146448, 0.8077588614146448, 0.8077588614146448, 0.8077588614146448, 0.8077588614146448]
[0.8619738791744108, 0.8619738791744108, 0.8619738791744108, 0.8619738791744108, 0.8619738791744108]
[0.5654755902680005, 0.5654755902680005, 0.5654755902680005, 0.5654755902680005, 0.5654755902680

In [35]:
sim

array([], dtype=float64)